# Create a movie with a sequence of moving stripe patterns

The movie consists of a sequence of patterns. Each pattern consists of an alternation of dark and light stripes moving at constant speed.

The movie is meant to be played on a tablet placed under the fish. Before creating a movie, edit the cell below to reflect the screen size and resolution of your device's screen.

In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
from os.path import exists, join
from glob import glob
import copy

# Screen size and resolution:
# Height: 1920 pixels / 24.46 cm 
# Width: 1200 pixels / 15.42 cm
# Pixels per cm: 1920 px / 24.46 cm ≈ 1200 px / 15.42 cm ≈ 78 px/cm
height_cm = 15.42
width_cm  = 24.46
cm_to_px  = 72

### Define the function that creates a single-pattern movie

In [ ]:
# Rectangular pattern: each stripe has uniform brightness; brightness changes
# abruptly when going from a dark stripe to a bright one.
def rect_pattern(i, color1, color2, period):
    return np.where((i%period)<period/2,color1,color2)

# Sine pattern: brightness changes smoothly, without abrupt change. Like a 
# blurred version of the rectangular pattern.
def sine_pattern(i, color1, color2, period):
    return 0.5 * (color2 - color1) * np.sin(2*np.pi*i/period) + 0.5 * (color1 + color2)

pattern_functions = { 'sine':sine_pattern, 'rect':rect_pattern }

def make_movie(output_file, duration, fps, color1, color2, period_cm, speed_cms, pattern):
    
    """
    output_file: path to the output video.
    duration: video duration, in s.
    fps: number of frames per second of the video.
    color1,color2: colors between which the pattern oscillates (integer between 0=black and 255=white).
    period: spatial period of the pattern, in cm.
    speed: speed at which the pattern translates, in cm/s.
    pattern_function: function that defines the desired spatial pattern. 
        Arguments: a pixel column index i, color1, color2, and period.
        Output: the shade of gray column i should have in the desired pattern.
    """
    
    height_px        = int(height_cm * cm_to_px)
    width_px         = int(width_cm * cm_to_px)
    period_px        = period_cm * cm_to_px
    speed_ppf        = speed_cms * cm_to_px / fps   # translation speed in pixels per frame
    n_frames         = int(duration * fps)          # total number of frames in the video
    
    codec            = 'mp4v' # 'H264' # 'X264' # 
    fourcc           = cv2.VideoWriter_fourcc(*codec)
#     fourcc           = 0x00000021
    video            = cv2.VideoWriter(output_file, fourcc, float(fps), (width_px, height_px), isColor=False)
#     print(video.isOpened())  # Check whether the video file was successfully initialized. 
    
    frame            = np.zeros((height_px,width_px), dtype = np.uint8)
    for n in range(n_frames):
        shift = n*speed_ppf
        for i in range(width_px):
            frame[:,i] = pattern(i-shift, color1, color2, period_px)
        video.write(frame)
    video.release()
    
    return

### Define the sequence of patterns that will form the full movie

The sequence is saved as a spreadsheet. Each row of the spreadsheet corresponds to a pattern and contains all the parameter values needed to create the movie for that specific pattern. The full movie will be formed by putting those single-pattern movies back to back.

Don't delete the spreadsheet; we'll need it to analyze the fish's behavior.

In [ ]:
# Lists of pattern parameters to include in the movie. 
period_list   = [2]        # Period of the light/dark pattern, in cm.
speed_list    = [1]        # Speed at which the strips move, in cm/s.
contrast_list = [255]      # Difference between the two stripe brightnesses. Max = 255 = black and white.
n_repeat      = 3          # Number of repetitions for each pattern. 1 repetition = moving right then moving left.

pattern_duration = 30       # Duration of each individual pattern, in seconds.

args0 = dict( pattern_name='rect', duration=pattern_duration, fps=30, # rest of the parameters 
             color1=0, color2=255, period_cm=None, speed_cms=None )

args_white = dict( pattern_name='rect', duration=pattern_duration, fps=30, # rest of the parameters 
                   color1=255, color2=255, period_cm=1, speed_cms=1 )

df = pd.DataFrame()
args_list = [args_white]
for period in period_list:
    for speed in speed_list:
        for contrast in contrast_list:
            for n in range(n_repeat):
                for direction in [1,-1]:
                    args = copy.deepcopy(args0)
                    args['period_cm'] = period
                    args['speed_cms'] = speed * direction
                    args['color1']    = int(127.5-contrast/2)
                    args['color2']    = args['color1']+contrast
                    args_list += [args]
args_list += [args_white]
            
df = pd.DataFrame(args_list)
df['pattern_id'] = -1
df['is_duplicate'] = df.duplicated()

for i in df.index:
    if not df.loc[i, 'is_duplicate']:
        df.loc[i, 'pattern_id'] = i
    else:
        current = df.loc[i].to_dict()
        del current['pattern_id']
        del current['is_duplicate']
        for j in range(i):
            previous = df.loc[j].to_dict()
            del previous['pattern_id']
            del previous['is_duplicate']
            if current == previous:
                df.loc[i, 'pattern_id'] = j
                break

del df['is_duplicate']
            
display(df)
df.to_excel('pattern_sequence.xlsx', index=False, engine='openpyxl')

### Load the pattern spreadsheet and create the full movie

In [ ]:
tmp_dir = './tmp'

def tmp_path(f):
    d = './tmp'
    if not os.path.exists(d):
        os.mkdir(d)
    return os.path.join(d,f)

df = pd.read_excel('pattern_sequence.xlsx')
# display(df)

conc_file = tmp_path('files.txt')
with open(conc_file, 'w') as f1:
    for i in df.index:
        args = df.loc[i].to_dict()
        f2   = f'{args["pattern_id"]}.mp4'
        if args['pattern_id']==i:
            args['pattern'] = pattern_functions[args['pattern_name']]
            del args['pattern_id']
            del args['pattern_name']
            make_movie(output_file=tmp_path(f2), **args)
        f1.write(f"\nfile '{f2}'")

# os.system(f'ffmpeg -y -f concat -safe 0 -i {conc_file} -c copy movie.mp4')
os.system(f'ffmpeg -y -f concat -safe 0 -i {conc_file} -c:v libx264 movie.mp4')